# MLP

### load data

In [118]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce
import operator 
import matplotlib.pyplot as plt
%matplotlib inline

def str_bool(x):
    if x == 'True':
        x=1
    elif x== 'False':
        x=0
    return x
        

In [119]:
gt=pd.read_csv('data/groundtruth_all.csv', index_col='tweet_id').drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)
print(gt.shape)
gt=gt.drop('id', axis =1)
gt.head()

(34511, 46)


/Users/valentine/anaconda/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,hashtags,text,lang,user_lang,retweeted,quoted_status,label,Hashtag topic 0,Hashtag topic 1,Hashtag topic 2,...,image_class,image_score,im_class_0,im_class_1,im_class_2,im_class_3,im_class_4,im_class_5,im_class_6,im_class_7
tweet_id,,,,,,,,,,,,,,,,,,,,,
55c272820a1776527f63ad53,wakeupman,Hahahahahah Mike t'es un ouuuuf 😂😂😂😂😂 @mrfligh...,fr,fr,False,NaN,1.0,0.125,0.125,0.125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c3bfea0a1776527f63df5b,NaN,@baboOu__ merciiiii beaucoup ! 😘😘❤️,fr,fr,False,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c5cf6f0a1776527f641ef3,NaN,Tu sais que ton père ne connaît rien à l'info ...,fr,fr,False,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c63c020a1776527f6430bd,NaN,J'ai envi de faire le Lucky Défi en vidéo ça v...,fr,fr,False,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c694b10a1776527f643f13,NaN,@Naif__acm @Eyaaaad @EH3H @nsr99nsr77 \n\nالصو...,ar,ar,False,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
#gt= pd.read_csv('data/groundtruth_hashtags_lang.csv',index_col=0 )

#check if NaN tweet_id 
#gt=gt[~gt['tweet_id'].isna()]
#gt=gt.set_index('tweet_id')

gt['quoted_status']= ~gt['quoted_status'].isnull()
gt[[ 'quoted_status']] = gt[[ 'quoted_status']].astype(int)

gt=gt.drop( [ 'lang', 'text', 'user_lang','retweeted'], axis =1)
#gt['retweeted'] = gt['retweeted'].apply(lambda x: str_bool(x))


lang_col = [col for col in gt.columns if col.startswith('lang_')]
user_lang_col= [col for col in gt.columns if col.startswith('user_lang_')]
hashtag_col= [col for col in gt.columns if col.startswith('Hashtag topic')] 
img_col= [col for col in gt.columns if col.startswith('im_class_')] + ['image_class', 'image_score'] 
topic_col= [col for col in gt.columns if col.startswith('t_topic_')]+['tweet_class']


gt[user_lang_col + ['other user lang']] = gt[user_lang_col + ['other user lang']].fillna(0).astype(int)
gt[lang_col + ['other lang']] = gt[lang_col + ['other lang']].fillna(0).astype(int)
gt[hashtag_col] = gt[hashtag_col].fillna(0).apply(lambda x: [str_bool(y) for y in x.values])
gt[hashtag_col] = gt[hashtag_col].astype(float)
print(gt.shape)
gt.head()

(34511, 41)


,hashtags,quoted_status,label,Hashtag topic 0,Hashtag topic 1,Hashtag topic 2,Hashtag topic 3,Hashtag topic 4,Hashtag topic 5,Hashtag topic 6,...,image_class,image_score,im_class_0,im_class_1,im_class_2,im_class_3,im_class_4,im_class_5,im_class_6,im_class_7
tweet_id,,,,,,,,,,,,,,,,,,,,,
55c272820a1776527f63ad53,wakeupman,0,1.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c3bfea0a1776527f63df5b,NaN,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c5cf6f0a1776527f641ef3,NaN,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c63c020a1776527f6430bd,NaN,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55c694b10a1776527f643f13,NaN,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
gt.columns

Index(['hashtags', 'quoted_status', 'label', 'Hashtag topic 0',
       'Hashtag topic 1', 'Hashtag topic 2', 'Hashtag topic 3',
       'Hashtag topic 4', 'Hashtag topic 5', 'Hashtag topic 6',
       'Hashtag topic 7', 'user_lang_fr', 'user_lang_en', 'user_lang_de',
       'user_lang_en-gb', 'user_lang_ar', 'lang_fr', 'lang_en', 'lang_und',
       'lang_es', 'lang_pt', 'other lang', 'other user lang', 'tweet_class',
       't_topic_0', 't_topic_1', 't_topic_2', 't_topic_3', 't_topic_4',
       't_topic_5', 't_topic_6', 'image_class', 'image_score', 'im_class_0',
       'im_class_1', 'im_class_2', 'im_class_3', 'im_class_4', 'im_class_5',
       'im_class_6', 'im_class_7'],
      dtype='object')

## Model with no hashtags topics, no images topics , no text topics

In [122]:
gt_20=gt[gt['hashtags'].isnull()]
gt_20=gt_20.drop(hashtag_col+ ['hashtags'] + img_col + topic_col, axis= 1)
gt_20.head()
print(gt_20.shape)

gt_20 = gt_20.sample(frac=1)
train_20=gt_20.iloc[0:int(round(0.5*gt_20.shape[0]))]
val_20=gt_20.iloc[ int(round(0.5*gt_20.shape[0])): int(round(0.75*gt_20.shape[0]))]
test_20=gt_20.iloc[int(round(0.75*gt_20.shape[0])): int(gt_20.shape[0])]

mlp_20= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_20.fit(train_20.drop('label',axis=1), train_20['label'])
score = mlp_20.score(test_20.drop('label', axis =1),test_20['label'])
print(score)

(26126, 14)
0.6269136558481323


## Model with hashtags

In [123]:
gt_hashtag=gt[~gt['hashtags'].isnull()]
gt_hashtag=gt_hashtag.drop(['hashtags'] + img_col + topic_col, axis= 1)
print(gt_hashtag.shape)
gt_hashtag.head()

gt_hashtag = gt_hashtag.sample(frac=1)
train_hashtag=gt_hashtag.iloc[0:int(round(0.5*gt_hashtag.shape[0]))]
val_hashtag=gt_hashtag.iloc[ int(round(0.5*gt_hashtag.shape[0])): int(round(0.75*gt_hashtag.shape[0]))]
test_hashtag=gt_hashtag.iloc[int(round(0.75*gt_hashtag.shape[0])): int(gt_hashtag.shape[0])]

mlp_hashtag= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_hashtag.fit(train_hashtag.drop('label',axis=1).fillna(0), train_hashtag['label'].fillna(0).astype(int))
score_hashtag = mlp_hashtag.score(test_hashtag.drop('label', axis =1).fillna(0),test_hashtag['label'].fillna(0).astype(int))
print(score_hashtag)

(8385, 22)
0.6894083969465649


## Model with images

In [129]:
gt_image=gt[~gt['image_class'].isnull()]
gt_image=gt_image.drop(['hashtags'] + hashtag_col + topic_col + ['image_class', 'image_score'], axis= 1)
print(gt_image.shape)
gt_image.head()

(3335, 22)


,quoted_status,label,user_lang_fr,user_lang_en,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,...,other lang,other user lang,im_class_0,im_class_1,im_class_2,im_class_3,im_class_4,im_class_5,im_class_6,im_class_7
tweet_id,,,,,,,,,,,,,,,,,,,,,
55ecd6310a177679d904ff7d,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0.000000,0.0,0.0,0.755072,0.0,0.0,0.0,0.000000
560ea35e0a17764c6ef1f147,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.939550
5652486f0a17761347b870d1,0,1.0,0,1,0,0,0,0,0,1,...,0,0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.916681
5654302a0a17761347b8b4a8,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.078300
56a38d8b0a1776437764bb49,0,1.0,1,0,0,0,0,0,0,0,...,1,0,0.386316,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [130]:
gt_image = gt_image.sample(frac=1)
train_image=gt_image.iloc[0:int(round(0.5*gt_image.shape[0]))]
val_image=gt_image.iloc[ int(round(0.5*gt_image.shape[0])): int(round(0.75*gt_image.shape[0]))]
test_image=gt_image.iloc[int(round(0.75*gt_image.shape[0])): int(gt_image.shape[0])]

mlp_image= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_image.fit(train_image.drop('label',axis=1).fillna(0), train_image['label'].fillna(0).astype(int))
score_image = mlp_image.score(test_image.drop('label', axis =1).fillna(0),test_image['label'].fillna(0).astype(int))
print(score_image)

0.5971223021582733


## Model with text topics

In [133]:
gt_topic=gt[~gt['tweet_class'].isnull()]
gt_topic=gt_topic.drop(['hashtags'] + hashtag_col + img_col +['tweet_class'], axis= 1)
print(gt_topic.shape)
gt_topic.head()

(34511, 21)


,quoted_status,label,user_lang_fr,user_lang_en,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,...,lang_pt,other lang,other user lang,t_topic_0,t_topic_1,t_topic_2,t_topic_3,t_topic_4,t_topic_5,t_topic_6
tweet_id,,,,,,,,,,,,,,,,,,,,,
55c272820a1776527f63ad53,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
55c3bfea0a1776527f63df5b,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
55c5cf6f0a1776527f641ef3,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
55c63c020a1776527f6430bd,0,1.0,1,0,0,0,0,1,0,0,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
55c694b10a1776527f643f13,0,1.0,0,0,0,0,1,0,0,0,...,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [134]:
gt_topic = gt_topic.sample(frac=1)
train_topic=gt_topic.iloc[0:int(round(0.5*gt_topic.shape[0]))]
val_topic=gt_topic.iloc[ int(round(0.5*gt_topic.shape[0])): int(round(0.75*gt_topic.shape[0]))]
test_topic=gt_topic.iloc[int(round(0.75*gt_topic.shape[0])): int(gt_topic.shape[0])]

mlp_topic= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_topic.fit(train_topic.drop('label',axis=1).fillna(0), train_topic['label'].fillna(0).astype(int))
score_topic = mlp_topic.score(test_topic.drop('label', axis =1).fillna(0),test_topic['label'].fillna(0).astype(int))
print(score_topic)

0.6336346777932313


## Model with all 

In [136]:
gt_all=gt[~gt['tweet_class'].isnull() & ~gt['hashtags'].isnull() & ~gt['image_class'].isnull() ]
gt_all=gt_all.drop(['hashtags']+ ['image_class', 'image_score'] + ['tweet_class'], axis= 1)
print(gt_all.shape)


gt_all = gt_all.sample(frac=1)
train_all=gt_all.iloc[0:int(round(0.5*gt_all.shape[0]))]
val_all=gt_all.iloc[ int(round(0.5*gt_all.shape[0])): int(round(0.75*gt_all.shape[0]))]
test_all=gt_all.iloc[int(round(0.75*gt_all.shape[0])): int(gt_all.shape[0])]

mlp_all= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_all.fit(train_all.drop('label',axis=1).fillna(0), train_all['label'].fillna(0).astype(int))
score_all = mlp_all.score(test_all.drop('label', axis =1).fillna(0),test_all['label'].fillna(0).astype(int))
print(score_all)

(1535, 37)
0.6432291666666666


/Users/valentine/anaconda/envs/ada/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Model with text topics and hashtags

In [137]:
gt_text_hashtags=gt[~gt['tweet_class'].isnull() & ~gt['hashtags'].isnull() ]
gt_text_hashtags=gt_text_hashtags.drop(['hashtags']+ img_col + ['tweet_class'], axis= 1)
print(gt_text_hashtags.shape)


gt_text_hashtags = gt_text_hashtags.sample(frac=1)
train_text_hashtags=gt_text_hashtags.iloc[0:int(round(0.5*gt_text_hashtags.shape[0]))]
val_text_hashtags=gt_text_hashtags.iloc[ int(round(0.5*gt_text_hashtags.shape[0])): int(round(0.75*gt_text_hashtags.shape[0]))]
test_text_hashtags=gt_text_hashtags.iloc[int(round(0.75*gt_text_hashtags.shape[0])): int(gt_text_hashtags.shape[0])]

mlp_text_hashtags= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_text_hashtags.fit(train_text_hashtags.drop('label',axis=1).fillna(0), train_text_hashtags['label'].fillna(0).astype(int))
score_text_hashtags = mlp_text_hashtags.score(test_text_hashtags.drop('label', axis =1).fillna(0),test_text_hashtags['label'].fillna(0).astype(int))
print(score_text_hashtags)

(8385, 29)
0.6755725190839694
